In [1]:
import sys
sys.path.append("/agrilearn_app/agrilearn/")
from agrilearn.crop_classification import s3_utils, str_utils
import boto3

2025-01-16 16:59:04.380615: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-16 16:59:04.398354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737046744.418924   16606 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737046744.425182   16606 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-16 16:59:04.445407: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
"""This script can be used for training a crop classification model"""
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from glob import glob
from geopandas import GeoDataFrame
from tqdm import tqdm
from eolearn.core import EOPatch
import warnings
warnings.filterwarnings("ignore") # Suprimindo todos os avisos
import pyogrio
pyogrio.core._register_drivers()
from agrilearn.crop_classification.cropclassifier import CropClassifier
from agrilearn.crop_classification.interface import CropClassificationInterface
from agrilearn.crop_classification import yaml_utils, processing

## Define PATHs

In [3]:
config = yaml_utils.load_config("/agrilearn_app/output/experiment/experiment_02.yaml")

### 1. Read Datasets

Os valores dessa documentação não batem
- https://pages.experian.local/pages/viewpage.action?pageId=1389466162

In [7]:
df = gpd.read_file(config['data']['geopackage_train_data_path'])

In [8]:
df.shape

(38192, 40)

In [9]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 38192 entries, 0 to 38191
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   monitoring_class            38192 non-null  object        
 1   period                      38192 non-null  object        
 2   fonte                       38192 non-null  object        
 3   cultura_2                   0 non-null      object        
 4   state                       38192 non-null  object        
 5   area                        38192 non-null  float64       
 6   meso                        38192 non-null  int64         
 7   obs_extra                   38192 non-null  object        
 8   eopath_location             38192 non-null  object        
 9   start_season                38192 non-null  object        
 10  end_season                  38192 non-null  object        
 11  peak_start                  38192 non-null  ob

### 2. Evaluate Geopackage
#### 2.1 Train, val, and test sets distribution

In [10]:
df['dataset_part'].value_counts(normalize=True)*100

dataset_part
train    69.978006
test     15.021470
val      15.000524
Name: proportion, dtype: float64

#### 2.2 Monitoring class distribution

In [11]:
df['monitoring_class'].value_counts(normalize=True)*100

monitoring_class
SOYBEAN    77.686426
CORN       14.950775
COTTON      4.291475
RICE        3.071324
Name: proportion, dtype: float64

In [12]:
report_train_val_test = df.groupby(['monitoring_class', 'dataset_part'], as_index=False)\
                            .agg({'length_of_season':'count'})\
                            .sort_values(['monitoring_class', 'length_of_season'], ascending=[False, False])
report_train_val_test

,monitoring_class,dataset_part,length_of_season
10,SOYBEAN,train,20763
9,SOYBEAN,test,4457
11,SOYBEAN,val,4450
7,RICE,train,820
6,RICE,test,178
8,RICE,val,175
4,COTTON,train,1146
5,COTTON,val,248
3,COTTON,test,245
1,CORN,train,3997


### 3. Evaluate Eopatch 

#### 3.1 Check predictions files to TEST Set